In [21]:
from card import card, Card, Suit
from typing import List, Optional, Union, cast, overload
import abc
from abc import abstractmethod
import random


Comparing class_init.py and card.py, class_init.py uses
complex __init__ in subclasses to simplify the factory
function. However, complex __init__() methods results in
a relatively minor improvement in the simplicity
 of a factory function. This is a common trade-off.

There's a trade-off that occurs between sophisticated
 __init__() methods and factory functions. It's often
 better to push complex constructors into factory functions.
 A factory function helps separate construction and initial
 state-from-state change or other processing concerns.

### using a list to create a collection of objs rather than a custom class
In this example, a list can be used to provide the container
for all the common function of a deck without creating a Deck class

Defining a class has the advantage of creating a simplified, implementation-free interface
to the object. In the case of the list example shown in the preceding code,
it's not clear how much simpler a Deck class would be.

In [6]:
d = [card(r + 1, s) for r in range(13) for s in iter(Suit)]
random.shuffle(d)
hand = [d.pop(), d.pop()]
hand

[<card.Card at 0x7f9428b0aa50>, <card.Card at 0x7f9428b0afd0>]

### Three designs to create and initialize collections in __init__
* Wrap: This design pattern surrounds an existing collection definition with a simplified interface. This is an example of the more general Facade design pattern.
* Extend: This design pattern starts with an existing collection class and extends it to add features.
* Invent: This is designed from scratch

### Wrapping a collection class
in the following code, we define Deck class to wrap the internal collection as a list object.
The pop() method of Deck simply delegates to the wrapped list object.

In [10]:
class Deck:
    def __init__(self) -> None:
        self._cards = [card(r + 1, s)
                       for r in range(13) for s in iter(Suit)]
        random.shuffle(self._cards)

    def pop(self) -> Card:
        return self._cards.pop()

d = Deck()
hand = [d.pop(), d.pop()]
hand

[<card.Card at 0x7f942883b490>, <card.Card at 0x7f942883bf90>]

Generally, a Facade design pattern or wrapper class contains methods that delegate the work to the underlying implementation class. This delegation can become wordy when a lot of features are provided. For a sophisticated collection, we may wind up delegating a large number of methods to the wrapped object.


### Extending a collection class
The following code extends list to create a collection.
An alternative to wrapping is to extend a built-in class. By doing this, we have the advantage of not having to reimplement the pop() method; we can simply inherit it.
The pop() method has an advantage in that it creates a class without writing too much code. In this example, extending the list class has the disadvantage that this provides many more functions than we truly need.


In [11]:
class Deck2(list):
    def __init__(self) -> None:
        super().__init__(
            card(r + 1, s) for r in range(13) for s in iter(Suit))
        random.shuffle(self)

d = Deck()
hand = [d.pop(), d.pop()]
hand

[<card.Card at 0x7f942883c790>, <card.FaceCard at 0x7f9428843710>]


In this case, we've initialized the list with Card instances. super().__init__() reaches up to the superclass initialization to populate our list object with an initial single deck of cards. After seeding the list, the initializer then shuffles the cards. The pop() method is directly inherited from list and works perfectly. Other methods inherited from the list class will also work.

While simpler, this exposes methods such as delete() and remove(). If these additional features are undesirable, a wrapped object might be a better idea.

### More requirements and design
In a casino, cards are often dealt from a shoe that has half a dozen decks of cards all mingled together. This additional complexity suggests that we need to build our own implementation of Deck and not simply use the list class directly. Additionally, a casino shoe is not dealt fully. Instead, a marker card is inserted. Because of the marker, some cards are effectively set aside and not used for play. These cards are said to be burned.

In [12]:
class Deck3(list):
    def __init__(self, decks: int = 1) -> None:
        super().__init__()
        for i in range(decks):
            self.extend( card(r + 1, s)
                    for r in range(13) for s in iter(Suit)
                            )
        random.shuffle(self)
        burn = random.randint(1, 52)
        for i in range(burn):
            self.pop()
deck = Deck3()
deck

Here, we used the __init__() method of the superclass to build an empty collection. Then, we used self.extend() to extend this collection with multiple 52- card decks. This populates the shoe. We could also use super().extend(), since we did not provide an overriding implementation in this class.
We could also carry out the entire task via super().__init__() using a more deeply nested generator expression, as shown in the following code snippet:

`
super().__init__(
     card(r + 1, s)
         for r in range(13)
             for s in iter(Suit)
                 for d in range(decks)
     )
`



### Complex composite objects
In this example, we have a self.dealer_card instance variable based on a parameter of the __init__() method. The self.cards instance variable, however, is not based on any parameter. This kind of initialization creates an empty collection. Note that the assignment to the self.cards variable requires a type hint to inform mypy of the expected contents of the self.cards collection.

In [13]:

class Hand:
    def __init__(self, dealer_card: Card) -> None:
        self.dealer_card: Card = dealer_card
        self.cards: List[Card] = []

    def hard_total(self) -> int:
        return sum(c.hard for c in self.cards)

    def soft_total(self) -> int:
        return sum(c.soft for c in self.cards)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__} {self.dealer_card} {self.cards}"

d = Deck()
h = Hand(d.pop())
h.cards.append(d.pop())
h.cards.append(d.pop())

This has the disadvantage of consisting of a long-winded sequence of statements to build an instance of a Hand object. It can become difficult to serialize the Hand object and rebuild it with an initialization as complex as this. Even if we were to create an explicit append() method in this class, it would still take multiple steps to initialize the collection.
The definition of the __repr__() method illustrates this problem. We can't provide a simple string representation that would rebuild the object. The typical use of __repr__() is to create a Pythonic view of the object's state, but, with such a complex initialization, there's no simple expression to represent it.
We could try to create a fluent interface, but that wouldn't really simplify things; it would merely mean a change in the syntax of the way that a Hand object is built. A fluent interface still leads to multiple method evaluations. When we take a look at the serialization of objects in part 2, Persistence and Serialization, we'd like an interface that's a single class-level function; ideally the class constructor.

The following code build composite class obj in one step in __init__()
The first positional argument value is assigned to the dealer_card parameter. The use of * with the cards parameter means that all of the remaining positional argument values are collected into a tuple and assigned to the cards parameter.

In [16]:
class Hand2:
    def __init__(self, dealer_card: Card, *cards: Card) -> None:
        self.dealer_card = dealer_card
        self.cards = list(cards)

    def card_append(self, card: Card) -> None:
        self.cards.append(card)

    def hard_total(self) -> int:
        return sum(c.hard for c in self.cards)

    def soft_total(self) -> int:
        return sum(c.soft for c in self.cards)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.dealer_card!r},*{self.cards})"

d = Deck()
p = Hand2(d.pop())
p.cards.append(d.pop())
p.cards.append(d.pop())
p


Hand2(<card.Card object at 0x7f94288652d0>,*[<card.Card object at 0x7f942884de50>, <card.Card object at 0x7f942884d290>])

This second example uses the *cards parameter to load a sequence of the Cards class in a single step:

In [18]:
d = Deck()
h = Hand2(d.pop(), d.pop(), d.pop())
h

Hand2(<card.Card object at 0x7f942884dc50>,*[<card.Card object at 0x7f942884d210>, <card.Card object at 0x7f9428b0a490>])

stateless objects without __init__()
The following is an example of a degenerate class that doesn't need an __init__() method. It's a common design pattern for Strategy objects. A Strategy object is plugged into some kind of master or owner object to implement an algorithm or decision. The Strategy object often depends on data in the master object; the Strategy object may not have any data of its own. We often design strategy classes to follow the Flyweight design pattern so we can avoid internal storage in the strategy instance. All values can be provided to a Strategy object as method argument values. In some cases, a strategy object can be stateless; in this instance, it is more a collection of method functions than anything else.
In the following examples, we'll show both stateless and stateful strategy class definitions. We'll start with the strategy for making some of the player decisions based on the state of the Hand object.
In this case, we're providing the gameplay decisions for a Player instance. The following is an example of a (dumb) strategy to pick cards and decline other bets:

Each method requires the current Hand object as an argument value. The decisions are based on the available information; that is, on the dealer's cards and the player's cards. The result of each decision is shown in the type hints as a Boolean value. Each method returns True if the player elects to perform the action.

In [ ]:
class GameStrategy:
    def insurance(self, hand: Hand) -> bool:
        return False

    def split(self, hand: Hand) -> bool:
        return False
    def double(self, hand: Hand) -> bool:
        return False
    def hit(self, hand: Hand) -> bool:
        return sum(c.hard for c in hand.cards) <= 17

We can build a single instance of this strategy for use by various Player instances, as shown in the following code snippet:
dumb = GameStrategy()
We can imagine creating a family of related strategy classes, each one using different rules for the various decisions a player is offered in Blackjack.

## other classes of the project:
a player has two strategies: one for betting and one for playing their hand. Each Player instance has a sequence of interactions with a larger simulation engine. We'll call the larger engine the Table class.
The Table class requires the following sequence of events by the Player instances:
1. The player must place an initial, or ante, bet based on the betting strategy.
2. The player will then receive a hand of cards.
3. If the hand is splittable, the player must decide whether to split it or not based on their game strategy. This can create additional Hand instances. In some casinos, the additional hands are also splittable.
4. For each Hand instance, the player must decide to hit, double, or stand based on their game strategy.
5. The player will then receive payouts, and they must update their betting strategy based on their wins and losses.
From this, we can see that the Table class has a number of API methods to receive a bet, create a Hand object, offer a split, resolve each hand, and pay off the bets. This is a large object that tracks the state of play with a collection of Players.
The following is the beginning of a Table class, which handles the bets and cards:

In [ ]:
class Table:
    def __init__(self) -> None:
        self.deck = Deck()

    def place_bet(self, amount: int) -> None:
        print("Bet", amount)

def get_hand(self) -> Hand2:
    try:
        self.hand = Hand2(self.deck.pop(), self.deck.pop(), self.deck.pop())
        self.hole_card = self.deck.pop()
    except IndexError:
      # Out of cards: need to shuffle and try again.
        self.deck = Deck()
        return self.get_hand()
    print("Deal", self.hand)
    return self.hand

def can_insure(self, hand: Hand) -> bool:
    return hand.dealer_card.insure

In order to interact properly and simulate realistic play, the Player class needs a betting strategy. The betting strategy is a stateful object that determines the level of the initial bet. Various betting strategies generally change a bet based on whether a game was a win or a loss.
Ideally, we'd like to have a family of BettingStrategy objects. Python has a module with decorators that allows us to create an abstract superclass

In [22]:

class BettingStrategy2(metaclass=abc.ABCMeta):
    @abstractmethod
    def bet(self) -> int:
        return 1

    def record_win(self):
        pass

    def record_loss(self):
        pass

NameError: name 'abc' is not defined

This has the advantage that it makes the creation of an instance of BettingStrategy2, or any subclass that failed to implement bet(), impossible. If we try to create an instance of this class with an unimplemented abstract method, it will raise an exception instead of creating an object.
And, yes, the abstract method has an implementation. It can be accessed via super().bet(). This allows a subclass to use the superclass implementation, if necessary.


### Multi-strategy __init__()
We may have objects that are created from a variety of sources. For example, we might need to clone an object as part of creating a memento, or freeze an object so that it can be used as the key of a dictionary or placed into a set; this is the idea behind the set and frozenset built-in classes.
We'll look at two design patterns that offer multiple ways to build an object. One design pattern uses a complex __init__() method with multiple strategies for initialization. This leads to designing the __init__() method with a number of optional parameters. The other common design pattern involves creating multiple static or class-level methods, each with a distinct definition.
Defining an overloaded __init__() method can be confusing to mypy, because the parameters may have distinct value types. This is solved by using the @overload decorator to describe the different assignments of types to the __init__() parameters. The approach is to define each of the alternative versions of
__init__() and decorate with @overload. A final version – without any decoration – defines the parameters actually used for the implementation.
The following is an example of a Hand3 object that can be built in either of the two ways:

In [23]:
class Hand3:
    @overload
    def __init__(self, arg1: "Hand3") -> None:
        ...

    @overload
    def __init__(self, arg1: Card, arg2: Card, arg3: Card) -> None:
        ...
    def __init__(self, arg1: Union[Card, "Hand3"],
                 arg2: Optional[Card] = None, arg3: Optional[Card] = None) -> None:
        self.dealer_card: Card
        self.cards: List[Card]
        if isinstance(arg1, Hand3) and not arg2 and not arg3: # Clone an existing hand
            self.dealer_card = arg1.dealer_card
            self.cards = arg1.cards
        elif (isinstance(arg1, Card)
                            and isinstance(arg2, Card)
                            and isinstance(arg3, Card)
                        ):
                           # Build a fresh, new hand.
            self.dealer_card = cast(Card, arg1)
            self.cards = [arg2, arg3]
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.dealer_card!r},*{self.cards})"

h = Hand3(deck.pop(), deck.pop(), deck.pop())
memento = Hand3(h)


In the first overloaded case, a Hand3 instance has been built from an existing Hand3 object. In the second case, a Hand3 object has been built from individual Card instances. The @overload decorator provides two alternative versions of the __init__() method. These are used by mypy to ensure this constructor is used properly. The undecorated version is used at runtime. It is a kind of union of the two overloaded definitions.
The @overload definitions are purely for mypy type-checking purposes. The non- overloaded definition of __init__() provides a hint for arg1 as union of either a Card object or a Hand3 object. The code uses the isinstance() function to decide which of the two types of argument values were provided. To be more robust, the if-elif statements should have an else: clause. This should raise a ValueError exception.
This design parallels the way a frozenset object can be built from individual items or an existing set object. We will look at creating immutable objects more in the next chapter. Creating a new Hand3 object from an existing Hand3 object allows us to create a memento of a Hand3 object using a construct such as the following code snippet:

In [26]:
h = Hand3(deck.pop(), deck.pop(), deck.pop())
memento = Hand3(h)
print(memento)

IndexError: pop from empty list